In [25]:
import os
import pandas as pd

In [26]:
from modules.config import DATADIR
from modules.utils import load_json, print_groupby

In [27]:
# Load factory contract addresses for each DAO framework
df_factories = pd.read_csv(os.path.join(DATADIR, 'framework_factory_contract_addresses.csv'), index_col=False)
df_factories.set_index('version', inplace=True)
df_factories['abis'] = None

# Load abis for each DAO framework
for version in df_factories.index:
    abis_full = load_json(os.path.join(DATADIR, f"trueblocks_framework_abis_{version}.json"))['data']
    abis = []
    for abi in abis_full:
        abiDict = {k: abi[k] for k in ['name', 'type']}
        abiDict['inputs'] = {v['name']: v['type'] for v in abi.get('inputs', {})}
        abis.append(abiDict)
    df_factories.at[version, 'abis'] = abis

print(df_factories.index[0])
df_factories['abis'][0]

aragon_0.8.1


[{'name': 'baseACL', 'type': 'function', 'inputs': {}},
 {'name': 'DeployEVMScriptRegistry',
  'type': 'event',
  'inputs': {'reg': 'address'}},
 {'name': 'newDAO', 'type': 'function', 'inputs': {'_root': 'address'}},
 {'name': 'DeployDAO', 'type': 'event', 'inputs': {'dao': 'address'}},
 {'name': 'regFactory', 'type': 'function', 'inputs': {}},
 {'name': 'baseKernel', 'type': 'function', 'inputs': {}}]

In [28]:
# Load in transaction data from all factory contracts by framework version
df_data = pd.DataFrame()
files = [f for f in os.listdir(DATADIR) if f.startswith('trueblocks_export')]
for f in files:
    version = "_".join(os.path.splitext(f)[0].split('_')[2:])
    dataDir = load_json(os.path.join(DATADIR, f))
    dataList = [{'index': k,
                 'framework': df_factories.at[version, 'framework'],
                 'version': version,
                 'functionName': v.get('articulatedTx', {}).get('name'),
                 'functionInputs': v.get('articulatedTx', {}).get('inputs'),
                 **v}
                for k, v in dataDir.items()]
    df_tmp = pd.DataFrame(dataList)
    df_data = df_data.append(df_tmp, ignore_index=True)

gb = df_data.groupby('version')
print(gb['index'].count())

version
aragon_0.6            258
aragon_0.7            242
aragon_0.8            150
aragon_0.8.1          150
aragon_court          311
daohaus_molochv2.1    242
daostack_alchemy      126
openlaw_tribute       108
Name: index, dtype: int64


In [29]:
# So what's in the transaction history for each framework?
for key, item in gb:
    print(key)
    df_group = pd.DataFrame(gb.get_group(key))
    df_create = df_group[df_group["contractAddress"] != "0x0"]
    print(f"{len(df_create.index)} DAOs created")
    
    df_update = df_group[df_group["contractAddress"] == "0x0"]
    gb_update = df_update.groupby(['functionName'])
    #print_groupby(gb_update[['version', 'date', 'to', 'functionName', 'functionInputs']])
    print(list(gb_update.groups.keys()))

aragon_0.6
8 DAOs created
['newAPM', 'newInstance']
aragon_0.7
5 DAOs created
['newDAO', 'newInstance']
aragon_0.8
3 DAOs created
['execTransaction', 'execute', 'newTokenAndBaseInstance', 'newTokenAndInstance', 'prepareInstance']
aragon_0.8.1
2 DAOs created
['execTransaction', 'execute', 'newTokenAndBaseInstance', 'prepareInstance']
aragon_court
3 DAOs created
['approveAndCall', 'changeModulesGovernor', 'contributeEth', 'contributeExternalToken', 'ensureCurrentTerm', 'forward', 'setModules', 'unstake', 'vote']
daohaus_molochv2.1
2 DAOs created
['execTransaction', 'execute', 'summonMoloch', 'summonMolochLLC']
daostack_alchemy
2 DAOs created
['addFounders', 'forgeOrg', 'setSchemes']
openlaw_tribute
16 DAOs created
['addAdapters', 'addExtension', 'configureExtension', 'createDao', 'updateAdapter']
